Drew Lickman\
CSCI 4820-001\
Project #2\
Due: 9/9/24

# N-Grams Algorithm

## Assignment Requirements:

### Input
---

- Two training data input files
    - CNN Stories
    - Shakespeare Plays
- Each line in the files are paragraphs, and paragraphs may contain multiple sentences

### Processing
---

- Text will be converted to lowercase during processing
- Extract n-grams in both methods
    - Sentence level
        - Paragraph will be sentence tokenized (NLTK sent_tokenize), then all sentences will be word tokenized (NLTK word_tokenize)
            - Resulting data will be augmented with \<s> and </s>
    - Paragraph level
        - Paragraph will be word tokenized (NLTK word_tokenize)
            - Resulting data will be augmented with \<s> and </s>
    - n-gram extraction should never cross over line boundaries
- The data structure used to hold tokens in each sentence should start with \<s> and end with </s>, according to the n-grams being processed
    - Higher order n-grams require more start symbol augments
- Unigrams, bigrams, trigrams, quadgrams will each be kept in separate data structures
    - Dictionaries, indexed by "context tuples" work well for this
- A parallel data structure should hold the counts of the tokens that immediately follow each n-gram context
    - These counts should be stored as probabilities by dividing by total count of tokens that appear after the n-gram context 
- Process both files first using sentence level, then followed by paragraph level

### Output
---

- Set NumPy seed to 0
- Print the count of extracted unigrams, bigrams, trigrams, and quadgrams (for each file)
- For each file, choose a random starting word from the unigram tokens (not </s>)
    - This random word will be used as the seed for generated n-gram texts
- For each gram:
    - Using the seed word (prefixed with \<s> as required) generate either 150 tokens or until </s> is generated
        - Do NOT continue after </s>
    - Each next token will be probabilistically selected from those that follow the context (if any) for hat n-gram
    - When working with higher order n-grams, use backoff when the context does not produce a token. Use the next lower n-gram

## Python Code

In [35]:
# Read text files and convert them to tokens

import numpy as np
import nltk as nltk

START = "<s>"
END = "<\s>"

# Dictionaries of n-grams
unigrams = {}
bigrams = {}
trigrams = {}
quadgrams = {}

sentences = []
with open("shakespeare.txt") as wordList:
    lines = wordList.readlines()
    for line in lines:
        line = line.lower()
        sentence = nltk.sent_tokenize(line)
        sentences.append(sentence) # Adds each sentence to the sentences array
        
for sent in sentences:
    for string in sent:
        token = nltk.word_tokenize(string) # Converts each word into a token
        print(token)



['o', 'thou', ',', 'my', 'lovely', 'boy', ',', 'who', 'in', 'thy', 'pow', "'", 'r']
['dost', 'hold', 'timeâ€™s', 'fickle', 'glass', 'his', 'sickle', 'hour', ',']
['who', 'hast', 'by', 'waning', 'grown', ',', 'and', 'therein', "show'st"]
['thy', 'lovers', 'withering', ',', 'as', 'thy', 'sweet', 'self', 'growâ€™stâ€', '”']
['in', 'nature', ',', 'sovereign', 'mistress', 'over', 'wrack', ',']
['as', 'thou', 'goest', 'onwards', 'still', 'will', 'pluck', 'thee', 'back', ',']
['she', 'keeps', 'thee', 'to', 'this', 'purpose', ',', 'that', 'her', 'skill']
['may', 'time', 'disgrace', ',', 'and', 'wretched', 'minute', 'kill', '.']
['yet', 'fear', 'her', ',', 'o', 'thou', 'minion', 'of', 'her', 'pleasure', ';']
['she', 'may', 'detain', 'but', 'not', 'still', 'keep', 'her', 'treasure', '.']
['her', 'audit', ',', 'though', 'delayed', ',', 'answered', 'must', 'be', ',']
['and', 'her', 'quietus', 'is', 'to', 'render', 'thee', '.']
